In [1]:
## Create a random array shape (6, 4)
import numpy as np

arr = np.random.rand(6, 4)
arr

array([[0.73639666, 0.166678  , 0.46304149, 0.40175927],
       [0.08921613, 0.76115958, 0.6780573 , 0.61689878],
       [0.43897931, 0.39329874, 0.44365883, 0.01974468],
       [0.46133171, 0.13587883, 0.06832373, 0.48647469],
       [0.03449602, 0.28625646, 0.1034433 , 0.39155329],
       [0.46474876, 0.21142336, 0.10480101, 0.56994037]])

In [8]:
np.concatenate((np.array([0] * arr.shape[0]).reshape(-1, 1), arr), axis=1)

array([[0.        , 0.73639666, 0.166678  , 0.46304149, 0.40175927],
       [0.        , 0.08921613, 0.76115958, 0.6780573 , 0.61689878],
       [0.        , 0.43897931, 0.39329874, 0.44365883, 0.01974468],
       [0.        , 0.46133171, 0.13587883, 0.06832373, 0.48647469],
       [0.        , 0.03449602, 0.28625646, 0.1034433 , 0.39155329],
       [0.        , 0.46474876, 0.21142336, 0.10480101, 0.56994037]])

In [10]:
np.repeat(0, 5).reshape(-1, 1)

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

# Define PAD tokens
PAD_ACTION = -1
PAD_DURATION = -1.0
PAD_DISTANCE = -1.0

class GenerativeLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_actions, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Separate output heads
        self.action_head = nn.Linear(hidden_size, num_actions)  # classification
        self.duration_head = nn.Linear(hidden_size, 1)          # regression
        self.distance_head = nn.Linear(hidden_size, 1)          # regression

    def forward(self, x, hidden=None):
        # x: [batch, seq_len, input_size]
        lstm_out, hidden = self.lstm(x, hidden)
        action_logits = self.action_head(lstm_out)             # [batch, seq_len, num_actions]
        duration = self.duration_head(lstm_out).squeeze(-1)    # [batch, seq_len]
        distance = self.distance_head(lstm_out).squeeze(-1)    # [batch, seq_len]
        return action_logits, duration, distance, hidden


In [ ]:
# Dummy training data (simulate variable-length sequences)
def generate_dummy_sequence(seq_len, input_size, num_actions):
    x = torch.randn(seq_len, input_size)
    action = torch.randint(0, num_actions, (seq_len,))
    duration = torch.rand(seq_len)
    distance = torch.rand(seq_len)
    return x, action, duration, distance

# Settings
batch_size = 3
input_size = 8
hidden_size = 32
num_actions = 5

# Create dummy variable-length data
sequences = [generate_dummy_sequence(torch.randint(4, 7, (1,)).item(), input_size, num_actions)
             for _ in range(batch_size)]

# Separate inputs and targets
inputs = [seq[0] for seq in sequences]
actions = [seq[1] for seq in sequences]
durations = [seq[2] for seq in sequences]
distances = [seq[3] for seq in sequences]

# Pad sequences
padded_inputs = pad_sequence(inputs, batch_first=True, padding_value=0.0)
padded_actions = pad_sequence(actions, batch_first=True, padding_value=PAD_ACTION)
padded_durations = pad_sequence(durations, batch_first=True, padding_value=PAD_DURATION)
padded_distances = pad_sequence(distances, batch_first=True, padding_value=PAD_DISTANCE)

# Create model
model = GenerativeLSTM(input_size=input_size, hidden_size=hidden_size, num_actions=num_actions)

# Forward pass
action_logits, duration_out, distance_out, _ = model(padded_inputs)

# Compute masks (True where valid)
mask = (padded_actions != PAD_ACTION)  # [batch, seq_len]

# Loss functions
action_loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_ACTION)
mse_loss = nn.MSELoss(reduction='none')

# Action loss (CrossEntropy expects shape [N, C] and targets [N])
action_loss = action_loss_fn(action_logits.view(-1, num_actions), padded_actions.view(-1))

# Duration & distance loss (masked MSE)
duration_loss = mse_loss(duration_out, padded_durations)
distance_loss = mse_loss(distance_out, padded_distances)

# Apply mask
duration_loss = (duration_loss * mask).sum() / mask.sum()
distance_loss = (distance_loss * mask).sum() / mask.sum()

# Total loss
total_loss = action_loss + duration_loss + distance_loss
print("Total Loss:", total_loss.item())


In [ ]:
def generate(model, start_input, seq_len):
    model.eval()
    input = start_input.unsqueeze(0).unsqueeze(1)  # [1, 1, input_size]
    hidden = None
    outputs = []

    for _ in range(seq_len):
        action_logits, duration_out, distance_out, hidden = model(input, hidden)
        action = torch.argmax(action_logits[:, -1], dim=-1).item()
        duration = duration_out[:, -1].item()
        distance = distance_out[:, -1].item()

        outputs.append((action, duration, distance))

        # Prepare next input (you may design your own input strategy here)
        next_input = torch.randn_like(start_input).unsqueeze(0).unsqueeze(1)
        input = next_input

    return outputs